# Snake deep q-learning


In [1]:
import pygame
import time
import random

# Initialisation de pygame
pygame.init()

# Couleurs
blanc = (255, 255, 255)
jaune = (255, 255, 102)
noir = (0, 0, 0)
rouge = (213, 50, 80)
vert = (0, 255, 0)
bleu = (50, 153, 213)

# Dimensions de la fenêtre
largeur_ecran = 600
hauteur_ecran = 400

# Taille du bloc (serpent)
taille_bloc = 10
vitesse_serpent = 15

# Configuration de la fenêtre
fenetre = pygame.display.set_mode((largeur_ecran, hauteur_ecran))
pygame.display.set_caption("Snake Game")

# Horloge pour gérer la vitesse du jeu
horloge = pygame.time.Clock()

# Font pour afficher le score
police = pygame.font.SysFont("bahnschrift", 25)

# Fonction pour afficher le score
def afficher_score(score):
    valeur = police.render("Score: " + str(score), True, jaune)
    fenetre.blit(valeur, [0, 0])

# Fonction principale du jeu
def jeu():
    game_over = False
    game_close = False

    # Position de départ du serpent
    x = largeur_ecran / 2
    y = hauteur_ecran / 2

    # Direction de déplacement
    x_changement = 0
    y_changement = 0

    # Corps du serpent
    serpent = []
    longueur_serpent = 1

    # Position initiale de la nourriture
    x_nourriture = round(random.randrange(0, largeur_ecran - taille_bloc) / 10.0) * 10.0
    y_nourriture = round(random.randrange(0, hauteur_ecran - taille_bloc) / 10.0) * 10.0

    while not game_over:

        while game_close:
            fenetre.fill(bleu)
            message = police.render("Perdu! Appuyez sur Q pour Quitter ou C pour Rejouer", True, rouge)
            fenetre.blit(message, [largeur_ecran / 6, hauteur_ecran / 3])
            afficher_score(longueur_serpent - 1)
            pygame.display.update()

            for event in pygame.event.get():
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_q:
                        game_over = True
                        game_close = False
                    if event.key == pygame.K_c:
                        jeu()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                game_over = True
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_LEFT:
                    x_changement = -taille_bloc
                    y_changement = 0
                elif event.key == pygame.K_RIGHT:
                    x_changement = taille_bloc
                    y_changement = 0
                elif event.key == pygame.K_UP:
                    y_changement = -taille_bloc
                    x_changement = 0
                elif event.key == pygame.K_DOWN:
                    y_changement = taille_bloc
                    x_changement = 0

        # Vérification des bordures
        if x >= largeur_ecran or x < 0 or y >= hauteur_ecran or y < 0:
            game_close = True
        x += x_changement
        y += y_changement
        fenetre.fill(noir)

        # Dessin de la nourriture
        pygame.draw.rect(fenetre, vert, [x_nourriture, y_nourriture, taille_bloc, taille_bloc])

        # Gestion du serpent
        tete_serpent = []
        tete_serpent.append(x)
        tete_serpent.append(y)
        serpent.append(tete_serpent)
        if len(serpent) > longueur_serpent:
            del serpent[0]

        # Vérification des collisions du serpent avec lui-même
        for bloc in serpent[:-1]:
            if bloc == tete_serpent:
                game_close = True

        # Dessin du serpent
        for bloc in serpent:
            pygame.draw.rect(fenetre, blanc, [bloc[0], bloc[1], taille_bloc, taille_bloc])

        afficher_score(longueur_serpent - 1)

        pygame.display.update()

        # Gestion de la nourriture
        if x == x_nourriture and y == y_nourriture:
            x_nourriture = round(random.randrange(0, largeur_ecran - taille_bloc) / 10.0) * 10.0
            y_nourriture = round(random.randrange(0, hauteur_ecran - taille_bloc) / 10.0) * 10.0
            longueur_serpent += 1

        horloge.tick(vitesse_serpent)

    pygame.quit()
    quit()

# Lancement du jeu
jeu()


pygame 2.5.2 (SDL 2.28.3, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


: 

In [4]:
import pygame
import random
import numpy as np

class SnakeEnv:
    def __init__(self):
        pygame.init()
        self.largeur_ecran = 600
        self.hauteur_ecran = 400
        self.taille_bloc = 10
        self.vitesse_serpent = 15
        self.fenetre = pygame.display.set_mode((self.largeur_ecran, self.hauteur_ecran))
        pygame.display.set_caption("Snake AI")
        self.horloge = pygame.time.Clock()

        # Définir les actions possibles : [0: Gauche, 1: Droite, 2: Haut, 3: Bas]
        self.actions = [0, 1, 2, 3]

        self.reset()

    def reset(self):
        """ Réinitialiser l'état du jeu """
        self.x = self.largeur_ecran / 2
        self.y = self.hauteur_ecran / 2
        self.x_changement = 0
        self.y_changement = 0
        self.serpent = [[self.x, self.y]]
        self.longueur_serpent = 1
        self.nourriture = self.generer_nourriture()
        self.score = 0
        return self.get_state()

    def generer_nourriture(self):
        """ Générer une nouvelle position de nourriture """
        return [round(random.randrange(0, self.largeur_ecran - self.taille_bloc) / 10.0) * 10.0,
                round(random.randrange(0, self.hauteur_ecran - self.taille_bloc) / 10.0) * 10.0]

    def get_state(self):
        """ Retourner un vecteur d'état représentant la situation actuelle """
        return np.array([self.x, self.y, self.nourriture[0], self.nourriture[1]])

    def step(self, action):
        """ Faire avancer le jeu d'un pas en fonction de l'action choisie """
        # Changer la direction en fonction de l'action
        if action == 0:  # Gauche
            self.x_changement = -self.taille_bloc
            self.y_changement = 0
        elif action == 1:  # Droite
            self.x_changement = self.taille_bloc
            self.y_changement = 0
        elif action == 2:  # Haut
            self.y_changement = -self.taille_bloc
            self.x_changement = 0
        elif action == 3:  # Bas
            self.y_changement = self.taille_bloc
            self.x_changement = 0

        # Mettre à jour la position du serpent
        self.x += self.x_changement
        self.y += self.y_changement
        self.serpent.append([self.x, self.y])

        if len(self.serpent) > self.longueur_serpent:
            del self.serpent[0]

        # Vérifier la collision avec les bords ou avec soi-même
        done = False
        if self.x >= self.largeur_ecran or self.x < 0 or self.y >= self.hauteur_ecran or self.y < 0:
            done = True
        for bloc in self.serpent[:-1]:
            if bloc == [self.x, self.y]:
                done = True

        # Gestion de la nourriture
        reward = 0
        if self.x == self.nourriture[0] and self.y == self.nourriture[1]:
            self.nourriture = self.generer_nourriture()
            self.longueur_serpent += 1
            reward = 10  # Récompense pour avoir mangé la nourriture
            self.score += 1

        # Faible pénalité pour chaque mouvement (pour encourager l'efficacité)
        reward -= 0.1

        return self.get_state(), reward, done

    def render(self):
        """ Dessiner l'état actuel du jeu """
        self.fenetre.fill((0, 0, 0))  # Fond noir
        # Dessiner la nourriture
        pygame.draw.rect(self.fenetre, (0, 255, 0), [self.nourriture[0], self.nourriture[1], self.taille_bloc, self.taille_bloc])
        # Dessiner le serpent
        for bloc in self.serpent:
            pygame.draw.rect(self.fenetre, (255, 255, 255), [bloc[0], bloc[1], self.taille_bloc, self.taille_bloc])

        pygame.display.update()
        self.horloge.tick(self.vitesse_serpent)




In [30]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from collections import deque
import random

class DQN:
    def __init__(self, env):
        self.env = env
        self.memory = deque(maxlen=2000)
        
        # Hyperparamètres de DQN
        self.gamma = 0.95  # Facteur de discount
        self.epsilon = 1.0  # Facteur d'exploration
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.batch_size = 64

        # Construction du modèle après initialisation des hyperparamètres
        self.model = self.build_model()

    def build_model(self):
        """Construit le modèle de réseau neuronal pour l'agent DQN"""
        model = Sequential()
        model.add(Dense(24, input_shape=(4,), activation='relu'))  # Correction de input_shape
        model.add(Dense(24, activation='relu'))
        model.add(Dense(4, activation='linear'))  # 4 sorties pour chaque action possible
        
        # Assure que la variable learning_rate est bien utilisée ici
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=self.learning_rate))  # Correction
        return model

    def remember(self, state, action, reward, next_state, done):
        """Enregistre une expérience dans la mémoire"""
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        """Choisit une action selon epsilon-greedy"""
        if np.random.rand() <= self.epsilon:
            return random.choice(self.env.actions)  # Exploration
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])  # Exploitation

    def replay(self):
        """Entraîne le modèle sur un échantillon de la mémoire"""
        if len(self.memory) < self.batch_size:
            return
        
        # Sélectionne un minibatch aléatoire
       


In [32]:
env = SnakeEnv()  # Remplace par ton environnement Snake
agent = DQN(env)

episodes = 1000  # Nombre d'épisodes d'entraînement
batch_size = 32  # Taille des lots pour l'entraînement

for e in range(episodes):
    state = env.reset()
    state = np.reshape(state, [1, 4])

    for time in range(500):  # Limite de temps pour chaque épisode
        action = agent.act(state)  # Choisir une action
        next_state, reward, done = env.step(action)  # Exécuter l'action
        reward = reward if not done else -10  # Pénalité en cas de défaite
        next_state = np.reshape(next_state, [1, 4])
        
        agent.remember(state, action, reward, next_state, done)  # Mémoriser la transition
        state = next_state
        
        if done:
            print(f"Episode {e}/{episodes} - Score : {time}")
            break
        
        if len(agent.memory) > batch_size:
            agent.replay()  # Entraîner le modèle avec un batch


Episode 0/1000 - Score : 332
Episode 1/1000 - Score : 292
Episode 2/1000 - Score : 485
Episode 3/1000 - Score : 198
Episode 4/1000 - Score : 437
Episode 8/1000 - Score : 295
Episode 13/1000 - Score : 316
Episode 14/1000 - Score : 164
Episode 16/1000 - Score : 318
Episode 19/1000 - Score : 363
Episode 20/1000 - Score : 230
Episode 21/1000 - Score : 268
Episode 22/1000 - Score : 396
Episode 24/1000 - Score : 189
Episode 27/1000 - Score : 361
Episode 32/1000 - Score : 440
Episode 38/1000 - Score : 218
Episode 43/1000 - Score : 305
Episode 44/1000 - Score : 494
Episode 45/1000 - Score : 343
Episode 46/1000 - Score : 318
Episode 48/1000 - Score : 385
Episode 52/1000 - Score : 112
Episode 53/1000 - Score : 265
Episode 54/1000 - Score : 405
Episode 57/1000 - Score : 184
Episode 58/1000 - Score : 343
Episode 60/1000 - Score : 180
Episode 63/1000 - Score : 301
Episode 64/1000 - Score : 225
Episode 68/1000 - Score : 124
Episode 72/1000 - Score : 396
Episode 75/1000 - Score : 392
Episode 78/1000 